> Project summary
- Import libraries
- 1: USE API TO FIND THE WEATHER FOR THE 35 CITIES (+7 DAYS) 
- 2: SCRAPPING BOOKING.com
- 3: CREATE CLEAN DATAFRAME WITH SCRAP BOOKING HOTELS DATAFRAME AND WEATHER DATAFRAME
- 4: Create S3 bucket | DATA LAKE
- 5: ETL process to clean data
- 6: Use RDS to use clean data | DATA WAREHOUSE
- 7: Define TOP5 cities based on average temperature for today + 7 days
- 8: Define TOP20 hotels based on the TOP5 cities
- 9: GRAPHICS | MAPBOX

# Import libraries

In [2]:
import pandas as pd

import requests
import os
import logging

import scrapy
from scrapy.crawler import CrawlerProcess

import plotly.express as px
import plotly.io as pio

import boto3
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker 

# 1: USE API TO FIND THE WEATHER FOR THE 35 CITIES (+7 DAYS)

In [ ]:
# Define variable for the cities
cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

In [ ]:
# Define access key to the API
APIkey = 'SECRET'

In [ ]:
# Define a list with lower city name & "+" between words
all_cities = [x.lower().replace(" ","+") for x in cities]

In [ ]:
# Define iteration to get the the wheather for the 35 cities from GPS coordonates
weather_list= []
for cities in all_cities:
    r = requests.get("https://nominatim.openstreetmap.org/search?q={}&format=json".format(cities))
    lat = r.json()[0]['lat']
    lon = r.json()[0]['lon']
    city = cities.lower().replace("+","")
    for i in range(8):
        r = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=hourly,minutely&appid={APIkey}")
        temp_day = r.json()['daily'][i]['temp']['day']
        wind_speed = r.json()['daily'][i]['wind_speed']
        main_weather = r.json()['daily'][i]['weather'][0]['main']
        day=i
        weather_list.append([city,temp_day,wind_speed,main_weather,day])

In [ ]:
# Create a dataframe from the list
df_all_cities = pd.DataFrame(weather_list, columns=['city','tempavg','windspeed','weather','day'])

# 2: SCRAPPING BOOKING.com

In [ ]:
# Define the spider to crawl pages and get the data needed
class bookingSpider(scrapy.Spider):
    # Define variables
    locs = ["MontSaintMichel","StMalo","Bayeux","LeHavre","Rouen","Paris","Amiens","Lille","Strasbourg","ChateauduHautKoenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","GorgesduVerdon","BormeslesMimosas","Cassis","Marseille","AixenProvence","Avignon","Uzes","Nimes","AiguesMortes","SaintesMariesdelamer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","LaRochelle"]
    name = "booking"
    init_url = dict() # set a dictionary for the urls to follow
    start_urls = ["https://www.booking.com/"]

    # Method to enter the cities in searching destination
    def parse(self, response):
        for loc in self.locs:
            # returns a new FormRequest object with its form field values pre-populated with those found in the HTML <form> element contained in the given response.
            yield scrapy.FormRequest.from_response(response,
                                                   formdata = {'ss': loc},
                                                   callback = self.after_search,
                                                   cb_kwargs = {'location':loc, 'page_no':0}
                                                   )

    # Method to find name, url, score and description of the hotels
    def after_search(self, response, location, page_no):
        # Condition to crawl pages
        if page_no == 0:
            self.init_url[location] = response.url
        
        cities = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for city in cities:
            name = city.css('div.fcab3ed991.a23c043802::text').get()
            url = city.css('a.e13098a59f').attrib['href']
            score = city.css('div.b5cd09854e.d10a6220b4::text').get()
            desc = city.css('div.d8eab2cf7f::text').get()
            
            dic = {'location': location,
                    'name': name,
                    'url': url,
                    'score':score,
                    'description': desc
                  }
            # initiate method for the parse_hotel GPS cordonnates 
            try: 
                yield response.follow(url = url,
                                      callback = self.GPS, 
                                      cb_kwargs = {'dic': dic}
                                     ) 
            except:
                dic['lat'] = None
                dic['lon'] = None
                yield dic
         
        if page_no <= 1: 
            # Create variable next_page by the url and end string to give to follow function below
            next_page = self.init_url[location]+"&offset="+str(25*(page_no+1)) 
            # follow method return a request instance to follow a link, here it is the next_page variable who contain previous url with a page tag at the end
            yield response.follow(next_page,
                                  callback = self.after_search,
                                  cb_kwargs = {'location':location, 'page_no':page_no+1}
                                 ) 

    # method to find GPS coordonates and split them in two variables lat & Lon        
    def GPS(self, response, dic): 
        try: 
            gps = response.css('a.jq_tooltip.loc_block_link_underline_fix.bui-link.show_on_map_hp_link.show_map_hp_link').attrib['data-atlas-latlng']   
        except:
            dic['lat']=None
            dic['lon']=None
            yield dic
            return None
        try:
            gps = gps.split(',')
            dic['lat']=float(gps[0])
            dic['lon']=float(gps[1])
            yield dic
        except:
            print("No way to split")
            dic['lat']=None
            dic['lon']=None
            yield dic
            
# Define a filename
filename = "scrap_booking.json"

# Remove previous file if it allready exist
if filename in os.listdir('src/'):
        os.remove('src/' + filename)

# Define the settings for the crawler process
process = CrawlerProcess(settings = {
    # simulate browser
    'USER_AGENT': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0', 
    # display regular errors
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'src/' + filename : {"format": "json"},
    }
})

# Initiate crawler
process.crawl(bookingSpider)
process.start()

# 3: Create merged data from scrapping

In [ ]:
# Create datafram from json hotels scrapping
df_hotels = pd.read_json('src/scrap_booking.json')

In [ ]:
# Removing lines with nan values from score column
df_hotels = df_hotels[df_hotels.score.isna() == False]

In [ ]:
# Keep the best hotels from each city
df_hotels = df_hotels[df_hotels.score > 8]

In [ ]:
# Rename location column to city
df_hotels.rename({'location': 'city'}, axis=1, inplace=True)

In [ ]:
# Lower city column
df_hotels.city = df_hotels.city.apply(lambda x: x.lower())

In [ ]:
# Create mergde data from both dataframes
merged_data = df_hotels.merge(df_all_cities, how='left', on='city')

In [ ]:
# Create CSV file from dataframe
merged_data.to_csv('Hotel-weather', encoding='utf-8', index=False)

# 4: Create S3 bucket | DATA LAKE

In [16]:
# Create instance session boto3
session = boto3.Session()

In [17]:
# Go to S3 aws 
s3 = session.resource("s3")

In [21]:
# Create s3 bucket 
bucket = s3.create_bucket(Bucket="booking-bucket-data", CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [ ]:
# Upload csv file to the bucket
bucket.put_object(Key="Hotel-weather.csv", Body = data)

In [34]:
# Download from s3 bucket
bucket.download_file('Hotel-weather.csv', 'Hotel-weather.csv')

# 5: ETL process to clean data

In [3]:
# Define dataframe from S3 csv
df=pd.read_csv('Hotel-weather.csv')

In [4]:
# Define Hotel dataframe from the previous
df_hotels = df[['city','name','url','score','description','lat','lon']].drop_duplicates(keep='first')

In [5]:
# Change temperature to degree Celcius
def degree(x):
    deg = x - 273.15
    return deg

In [ ]:
# Create dataframe weather from first dataframe and apply degree function
df_weather = df[['city','lat','lon','tempavg','windspeed','weather','day']]
df_weather.tempavg = df_weather.tempavg.apply(degree)

# 6: Use RDS to use clean data | DATA WAREHOUSE

In [7]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker 

In [8]:
# Define database connection variables
DBHOST = "database-booking-kg.c9wr39q76493.eu-west-3.rds.amazonaws.com"
DBUSER = "SECRET"
DBPASS = "SECRET"
DBNAME = "postgres"

In [9]:
# Connect to my postgresSQL database
engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)

In [10]:
# Initialize a sessionmaker to talk to our database
Session = sessionmaker(bind = engine)
# Instanciate session 
session = Session()

In [11]:
# Transform the dataframes to SQL
df_weather.to_sql("df_weather_sql", engine)
df_hotels.to_sql("df_hotels_sql", engine)

2022-07-15 09:32:48,431 INFO sqlalchemy.engine.Engine select version()
2022-07-15 09:32:48,432 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 09:32:48,440 INFO sqlalchemy.engine.Engine select current_schema()
2022-07-15 09:32:48,441 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 09:32:48,459 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-07-15 09:32:48,459 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 09:32:48,465 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-15 09:32:48,466 INFO sqlalchemy.engine.Engine [generated in 0.00049s] {'name': 'df_weather_sql'}
2022-07-15 09:32:48,488 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-15 09:32:48,488 INFO sqlalchemy.engine.Engine 
CREATE TABLE df_weather_sql (
	index BIGINT, 
	city TEXT, 
	lat FLOAT(53), 
	lon FLOAT(53), 
	tempavg FLOAT(53), 
	windspeed FLOAT(53), 
	

In [12]:
# Create a statement
# SQL base selection for the next data analysis team
Selection_weather = text("SELECT * FROM df_weather_sql")
df_weather = pd.read_sql(Selection_weather, engine)

Selection_hotels = text("SELECT * FROM df_hotels_sql")
df_hotels = pd.read_sql(Selection_hotels, engine)

2022-07-15 09:33:00,332 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-15 09:33:00,332 INFO sqlalchemy.engine.Engine [cached since 11.87s ago] {'name': 'SELECT * FROM df_weather_sql'}
2022-07-15 09:33:00,342 INFO sqlalchemy.engine.Engine SELECT * FROM df_weather_sql
2022-07-15 09:33:00,342 INFO sqlalchemy.engine.Engine [generated in 0.00046s] {}
2022-07-15 09:33:00,498 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-15 09:33:00,498 INFO sqlalchemy.engine.Engine [cached since 12.03s ago] {'name': 'SELECT * FROM df_hotels_sql'}
2022-07-15 09:33:00,511 INFO sqlalchemy.engine.Engine SELECT * FROM df_hotels_sql
2022-07-15 09:33:00,511 INFO sqlalchemy.engine.Engine [generated in 0.00072s] {}


# 7: Define TOP5 cities based on average temperature for today + 7 days

In [11]:
# Create dataframe with descending value means from average temperature column
df_weather_mean = df_weather.groupby(by='city', as_index = False).mean().sort_values(by = 'tempavg',ascending = False)

In [12]:
# Create new dataframe with only five first rows
df_weather_top5 = df_weather_mean.head(5)

In [13]:
# Create list of city with higher average temperature for the 7 days
Top5_cities = df_weather_top5.city.tolist()

# 8: Define TOP20 hotels based on the TOP5 cities

In [14]:
# Create dataframe from the top5 list of best cities
df_hotels_best = df_hotels[df_hotels['city'].isin(Top5_cities)]

In [15]:
# Create dataframe from previous sorting values from city and score columns
df_hotels_best_score = df_hotels_best.sort_values(by = ['city','score'],ascending = False)

In [16]:
# Create grouped dataframe from previous by the city column
grouped_city_df = df_hotels_best_score.groupby('city')

In [17]:
# Create dataframe with only 20 first rows from preivou grouped dataframe
Top20_hotels = grouped_city_df.head(20)

# 9: GRAPHICS | MAPBOX

In [19]:
# Display weather map
fig = px.scatter_mapbox(df_weather.sort_values('day'),
                        title='Weather map',
                        lat = 'lat',
                        lon = 'lon',
                        color = 'tempavg',
                        color_continuous_scale = px.colors.diverging.balance,
                        size = 'windspeed',
                        size_max = 30,
                        hover_name = 'city',
                        animation_frame = 'day'
                        )
fig.update_layout(width = 1000,
                  height = 800,
                  title_x = 0.5, 
                  template = 'plotly_dark',
                  mapbox = {"style": "carto-darkmatter", "center": {"lon": 2, "lat" : 47}, "zoom": 4.8},
                  margin = {"l": 0, "r": 0, "b": 0, "t": 80},
                  )

fig.show()

In [20]:
# Display Top 20 hotels for the top 5 cities
fig = px.scatter_mapbox(Top20_hotels,
                        title = 'Best Hotels',
                        lat = 'lat',
                        lon = 'lon',
                        color = 'score',
                        size = 'score',
                        color_continuous_scale = px.colors.diverging.Tropic,
                        size_max = 20,
                        hover_name = 'name',
                        hover_data = {'description': True}
                        )
fig.update_layout(width = 1000,
                  height = 800,
                  title_x = 0.5, 
                  template='plotly_dark',
                  mapbox = {"style": "carto-darkmatter", "center": {"lon": 2.6, "lat" : 43.5}, "zoom": 7},
                  margin = {"l": 0, "r": 0, "b": 0, "t": 80},
                  )
fig.show()